### Time Series Forecasting using LSTM

In [ ]:
# Importing libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import os

mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False



In [ ]:
orig_df = pd.read_csv('/Users/faymajidelhassan/Downloads/Master project /Data/Weather/measurements/Combinedweather_measurements.csv') 
df = orig_df.copy() 
print(f'Size of the dataset: {df.shape} \n')  
print() 
display(df.head(5))

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)


In [ ]:
df.isnull().sum()
# df=df.fillna(df.mean())

In [ ]:
# df=df.fillna(df.mean())
df = df.apply(lambda col: col.fillna(col.mean()), axis=0)

df.isnull().sum()
df.head()

Observations:
1) One reading evrry 10 mins (from datatime column time diff for every record )
2) 1day = 6*24 = 144 readings
Task : Forecasting Temperature(in degree ) in future 




In [ ]:
## using univariate feature(Only temperature for given time)

uni_data = df['temperature']
# uni_data.index = df['timestamp']
uni_data.head()

In [ ]:
uni_data.plot()

In [ ]:
uni_data = uni_data.values

In [ ]:
uni_data.shape

In [ ]:
## train test split for simple time series moving window average
train_split = int(len(uni_data) * 0.8)#205082 #43228
tf.random.set_seed(13)

### standardize data
uni_data_mean = uni_data[:train_split].mean()
uni_data_std = uni_data[:train_split].std()
uni_data  = (uni_data - uni_data_mean)/ uni_data_std

print(type(uni_data))


Moving Window Average


1.   Given last 20 values of observations(temp) , predict next observation
2.   MWA: predict== AVG(last 20 values)




In [ ]:
## utility functions

## funtion to create data for univariate forecasting

def univariate_data(dataset, start_idx , end_idx , history_size, target_size):
  data = []
  labels = []
  start_idx  = start_idx + history_size
  if end_idx is None:
    end_idx = len(dataset)- target_size
  for i in range(start_idx , end_idx):
    idxs = range(i-history_size , i)
    data.append(np.reshape(dataset[idxs] , (history_size, 1))) ### reshape data
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

uni_data_history = 20   ## last 50 values
uni_data_future = 0     ## future data

x_train_uni , y_train_uni = univariate_data(uni_data , 0 , train_split , uni_data_history , uni_data_future)

x_val_uni , y_val_uni = univariate_data(uni_data , train_split , None ,uni_data_history , uni_data_future)

In [ ]:
x_train_uni

In [ ]:
print(x_train_uni.shape , y_train_uni.shape)
print(x_val_uni.shape , y_val_uni.shape)

In [ ]:
print('Single window of history data' , x_train_uni[0])

print('Target Temperature to predict ' , y_train_uni[0])


In [ ]:
### fucntion to create time steps
def create_time_steps(length):
  return list(range(-length,0))

### function to plot time series data

def plot_time_series(plot_data, delta , title):
  labels = ["History" , 'True Future' , 'Model Predcited']
  marker = ['.-' , 'rx' , 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])

  if delta:
    future = delta
  else:
    future = 0
  plt.title(title)
  for i , x in enumerate(plot_data):
    if i :
      plt.plot(future , plot_data[i] , marker[i], markersize = 10 , label = labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label = labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5) *2])

  plt.xlabel('Time_Step')
  return plt

plot_time_series([x_train_uni[0] , y_train_uni[0]] , 0 , 'Sample Example')

In [ ]:
i = 20
plot_time_series([x_train_uni[i], y_train_uni[i]] , 0 , 'Sample Example')

In [ ]:
### Moving window average

def MWA(history):
  return np.mean(history)




In [ ]:
i = 20
plot_time_series([x_train_uni[i] , y_train_uni[i] , MWA(x_train_uni[i])] , 0 , 'MWA predicted')

Univariate time-series forecasting


*   Only single feature as temperature(historical data)
*   Task:  Given last 20 observations(history) , predict next temperature value 



In [ ]:
## prepare tensorflow dataset
batch_size = 256
buffer_size = 10000

train_uni = tf.data.Dataset.from_tensor_slices((x_train_uni , y_train_uni))
train_uni = train_uni.cache().shuffle(buffer_size).batch(batch_size).repeat()

val_uni = tf.data.Dataset.from_tensor_slices((x_val_uni , y_val_uni))
val_uni = val_uni.cache().shuffle(buffer_size).batch(batch_size).repeat()

print(train_uni)
print(val_uni)

In [ ]:
## Define LSTM model 

lstm_model = tf.keras.models.Sequential([tf.keras.layers.LSTM(16 , input_shape = x_train_uni.shape[-2:]), 
                                         tf.keras.layers.Dense(1)])

lstm_model.compile(optimizer = 'adam', loss = 'mae')

steps = 200

EPOCHS =10

lstm_model.fit(train_uni , epochs = EPOCHS, steps_per_epoch = steps ,
               validation_data = val_uni, validation_steps = 50)




In [ ]:
for i , j in val_uni.take(5):
  plot = plot_time_series([i[0].numpy() , j[0].numpy() , lstm_model.predict(i)[0]] ,0 , 'LSTM UNIVARIATE')
  plot.show()

Multivariate  and Single step Forecasting


*   Task: Given 3 features(temp , pressure , and density) at each time step can we predict the temp in future at single time step




In [ ]:
## features 

features_6 = ['temperature', 'humidity', 'pressure', 'global_irradiance', 'direct_irradiance', 'diffuse_irradiance','precipitation']

features = df[features_6]
# features.index = df['timestamp']
features.head()



In [ ]:
features.isnull().sum()

In [ ]:
features.plot(subplots=True)

In [ ]:
### standardize data
dataset = features.values
# dataset = np.array(features)

data_mean = dataset[:train_split].mean(axis =0)

data_std = dataset[:train_split].std(axis = 0)

dataset = (dataset - data_mean)/data_std



In [ ]:
### create mutlivariate data

def mutlivariate_data(dataset , target , start_idx , end_idx , history_size , target_size,
                      step ,  single_step = False):
  data = []
  labels = []
  start_idx = start_idx + history_size
  if end_idx is None:
    end_idx = len(dataset)- target_size
  for i in range(start_idx , end_idx ):
    idxs = range(i-history_size, i, step) ### using step
    data.append(dataset[idxs])
    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data) , np.array(labels)



In [ ]:
# ### generate multivariate data

history = 720
future_target = 72
STEP = 6

x_train_ss , y_train_ss = mutlivariate_data(dataset , dataset[:, 1], 0, train_split, history,
                                            future_target, STEP , single_step = True)

x_val_ss , y_val_ss = mutlivariate_data(dataset , dataset[:,1] , train_split , None , history ,
                                        future_target, STEP, single_step = True)



print(x_train_ss.shape, y_train_ss.shape)
print(x_val_ss.shape, y_val_ss.shape)


In [ ]:
## tensorflow dataset

train_ss = tf.data.Dataset.from_tensor_slices((x_train_ss, y_train_ss))
train_ss = train_ss.cache().shuffle(buffer_size).batch(batch_size).repeat()

val_ss = tf.data.Dataset.from_tensor_slices((x_val_ss, y_val_ss))
val_ss = val_ss.cache().shuffle(buffer_size).batch(batch_size).repeat()

print(train_ss)
print(val_ss)



In [ ]:
### Modelling using LSTM
from tensorflow.keras import layers, models, optimizers, callbacks
# Cosine Learning Rate Schedule
initial_learning_rate = 0.001
decay_steps = 1000  # Number of steps after which learning rate is decayed
alpha = 0.0  # Minimum learning rate value as a fraction of the initial learning rate

cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    alpha=alpha
)
from keras.callbacks import EarlyStopping
callbacks = EarlyStopping(
    patience = 5 , 
    restore_best_weights = True , 
    monitor = 'val_loss'
)
# lstm_model = tf.keras.models.Sequential([tf.keras.layers.LSTM(16 , input_shape = x_train_uni.shape[-2:]), 
#                                          tf.keras.layers.Dense(1)])
single_step_model = tf.keras.models.Sequential()

single_step_model.add(tf.keras.layers.LSTM(32,input_shape = x_train_ss.shape[-2:]))
# single_step_model.add(layers.LayerNormalization())
# single_step_model.add(tf.keras.layers.LSTM(64,return_sequences=False))
# single_step_model.add(layers.LayerNormalization())/
# single_step_model.add(tf.keras.layers.Dense(2, activation="relu"))
single_step_model.add(tf.keras.layers.Dense(1))
single_step_model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'mae')
single_step_model_history = single_step_model.fit(train_ss, epochs = EPOCHS ,
                                                  steps_per_epoch =steps,verbose=1, validation_data = val_ss,
                                                  validation_steps = 50,callbacks=[callbacks])


In [ ]:
directory = '/Users/faymajidelhassan/Downloads/Master project /CODE/EDA/Saved_models'
filename = 'Lstm_single_step_model_measure+precip.h5'
model_path = f'{directory}/{filename}'
single_step_model.save(model_path)

In [ ]:
## plot train test loss 

def plot_loss(history , title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))
  plt.figure()
  plt.plot(epochs, loss , 'b' , label = 'Train Loss')
  plt.plot(epochs, val_loss , 'r' , label = 'Validation Loss')
  plt.title(title)
  plt.legend()
  plt.grid()
  plt.show()

plot_loss(single_step_model_history , 'Single Step Training and validation loss')

In [ ]:
for x, y in val_ss.take(5):
  plot = plot_time_series([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
  plot.show()

Multi-variate & multi-step forecasting
-> Generate multiple future values of temperature

In [ ]:
future_target = 72 # 72 future values
x_train_multi, y_train_multi = mutlivariate_data(dataset, dataset[:, 1], 0,
                                                 train_split, history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = mutlivariate_data(dataset, dataset[:, 1],
                                             train_split, None, history,
                                             future_target, STEP)

print(x_train_multi.shape)
print(y_train_multi.shape)




In [ ]:
# TF DATASET

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(buffer_size).batch(batch_size).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(batch_size).repeat()

In [ ]:
#plotting function
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)
  plt.grid()
  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'b-',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'r-',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()
  


for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

In [ ]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
# multi_step_model.add(tf.keras.layers.LSTM(4,return_sequences=False, activation='relu'))
multi_step_model.add(tf.keras.layers.LSTM(16,activation='relu')) # for 72 output
multi_step_model.add(tf.keras.layers.Dense(72)) # for 72 outputs

multi_step_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cosine_decay,clipvalue=1.0), loss='mae')
multi_step_model.summary()
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=steps,
                                          validation_data=val_data_multi,
                                          validation_steps=50,callbacks=[callbacks])





In [ ]:


# Create a new model that outputs the features from the second LSTM layer
feature_extractor = tf.keras.Model(inputs=multi_step_model.input,
                                   outputs=multi_step_model.get_layer(index=0).output)  # LSTM layer at index 1

# Summary of the feature extractor model
feature_extractor.summary()

# Example data for which you want to extract features (replace with actual data)
x_val_example, y_val_example = next(iter(val_data_multi))  # Get a batch of validation data

# Extract LSTM features
lstm_features = feature_extractor.predict(x_val_example)

# Print the shape of the extracted features
print(f'LSTM features shape: {lstm_features.shape}')


In [ ]:
plot_loss(multi_step_history, 'Multi-Step Training and validation loss')


In [ ]:
for x, y in val_data_multi.take(5):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

In [ ]:
directory = '/Users/faymajidelhassan/Downloads/Master project /CODE/EDA/Saved_models'
filename = 'Lstm_multi_step_model_measure+precip.h5'
model_path = f'{directory}/{filename}'
multi_step_model.save(model_path)

In [ ]:
import xgboost as xgb

# Define and train XGBoost model

# Train XGBoost model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
model_xgb.fit(x_train_multi.reshape(x_train_multi.shape[0], -1), y_train_multi[:, -1])  # Using the last value of the future target for training


In [ ]:
# Evaluate LSTM model
mae_lstm_single = single_step_model.evaluate(val_ss, steps=100)
mae_lstm_multi=multi_step_model.evaluate(val_data_multi, steps=100)


# Evaluate XGBoost model
xgb_forecast = model_xgb.predict(x_val_multi.reshape(x_val_multi.shape[0], -1))
mae_xgb = np.mean(np.abs(xgb_forecast - y_val_multi[:, -1]))

# Calculate MAE for SARIMAX model
# mae_sarimax = np.mean(np.abs(sarimax_forecast - df['temperature'][train_split:]))

print(f"MAE for single LSTM: {mae_lstm_single}")
print(f"MAE for LSTM: {mae_lstm_multi}")
print(f"MAE for XGBoost: {mae_xgb}")
# print(f"MAE for SARIMAX: {mae_sarimax}")


In [ ]:
def plot_forecast(true_values, lstm_pred, xgb_pred, ensemble_pred):
    plt.figure(figsize=(12, 6))
    plt.plot(true_values, label='True Values')
    plt.plot(lstm_pred, label='LSTM Predictions')
    plt.plot(xgb_pred, label='XGBoost Predictions')
    plt.plot(ensemble_pred, label='Ensemble Predictions', linestyle='--')
    plt.legend()
    plt.show()

In [ ]:
import numpy as np

# Example data
true_values = np.random.rand(100)
# lstm_pred = np.random.rand(100)
# xgb_pred = np.random.rand(100)
ensemble_pred = np.random.rand(100)

# Plot the forecasts
plot_forecast(true_values, lstm_pred, xgb_pred, ensemble_pred)


In [ ]:
# Define parameters
from sklearn.preprocessing import StandardScaler
# history = 720
# future_target = 72
# STEP = 6
# BATCH_SIZE = 128
# BUFFER_SIZE = 10000
# EPOCHS = 10
# steps_per_epoch = 200
# train_split = int(len(dataset) * 0.7)

# # Fit StandardScaler on training data
# scaler = StandardScaler()
# scaler.fit(dataset[:train_split])

# # Transform the entire dataset
dataset_scaled = dataset

# Function to create multivariate data
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i:i + target_size])

    return np.array(data), np.array(labels)

# Create the multivariate data with the scaled dataset
x_train_ss, y_train_ss = multivariate_data(dataset_scaled, dataset_scaled[:, 1], 0, train_split, history, future_target, STEP, single_step=True)
x_val_ss, y_val_ss = multivariate_data(dataset_scaled, dataset_scaled[:, 1], train_split, None, history, future_target, STEP, single_step=True)

# Check shapes
print(x_train_ss.shape, y_train_ss.shape)
print(x_val_ss.shape, y_val_ss.shape)

# Define the dataset for training and validation
train_ss = tf.data.Dataset.from_tensor_slices((x_train_ss, y_train_ss))
val_ss = tf.data.Dataset.from_tensor_slices((x_val_ss, y_val_ss))

# Batch the datasets
train_ss = train_ss.cache().shuffle(buffer_size).batch(batch_size).repeat()
val_ss = val_ss.batch(batch_size).repeat()


In [ ]:
# Define the GRU model
def create_gru_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.GRU(32,return_sequences=True),
        tf.keras.layers.GRU(16,activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# Assume x_train_ss.shape is (num_samples, time_steps, features)
input_shape = input_shape=x_train_multi.shape[-2:]
gru_model = create_gru_model(input_shape)

# Train the GRU model
gru_model_history = gru_model.fit(train_ss, epochs=EPOCHS, steps_per_epoch=steps, validation_data=val_ss, validation_steps=50)

# Evaluate GRU model
mae_gru_single = gru_model.evaluate(val_ss, steps=100)
print("Mean Absolute Error for GRU model:", mae_gru_single)


In [ ]:
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dense, Dropout, Flatten, Input
import tensorflow as tf

# Transformer Block for time series forecasting
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"), 
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build Transformer model
def build_transformer_model(input_shape, embed_dim):
    inputs = Input(shape=input_shape)
    # Add a Dense layer to project input to the embedding dimension
    x = Dense(embed_dim)(inputs)
    x = TransformerBlock(embed_dim=embed_dim, num_heads=2, ff_dim=32)(x)
    x = Flatten()(x)
    x = Dense(20, activation="relu")(x)
    outputs = Dense(1)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


# Build and compile Transformer model
input_shape = x_train_multi.shape[-2:]  # Correct input shape
embed_dim = 32  # Embedding dimension
transformer_model = build_transformer_model(input_shape, embed_dim)
transformer_model.compile(optimizer='adam', loss='mae')

# Train Transformer model
transformer_model_history = transformer_model.fit(train_ss, epochs=EPOCHS, steps_per_epoch=steps,
                                                  validation_data=val_ss, validation_steps=50)

# Evaluate Transformer model
mae_transformer_single = transformer_model.evaluate(val_ss, steps=100)
print("Mean Absolute Error for Transformer model:", mae_transformer_single)



In [ ]:
import matplotlib.pyplot as plt

def plot_time_series(data, title):
    plt.figure(figsize=(10, 6))
    time_steps = range(len(data[0]))  # History length
    plt.plot(time_steps, data[0], label='History')  # Plot the historical data
    plt.plot(len(data[0]), data[1], 'bo', label='True Future')  # Plot the true future value as a single point
    plt.plot(len(data[0]), data[2], 'ro', label='Predicted Future')  # Plot the predicted future value as a single point
    plt.title(title)
    plt.legend()
    return plt

# Plotting function for Transformer predictions
for x, y in val_ss.take(5):
    prediction = transformer_model.predict(x)
    plot = plot_time_series([x[0].numpy(), y[0].numpy(), prediction[0]], 'Transformer UNIVARIATE')
    plot.show()


In [ ]:
# Evaluate GRU model
mae_gru_single = gru_model.evaluate(val_ss, steps=100)
print(f"MAE for GRU: {mae_gru_single}")

# Evaluate Transformer model
mae_transformer_single = transformer_model.evaluate(val_ss, steps=100)
print(f"MAE for Transformer: {mae_transformer_single}")


# Generate predictions for each model
pred_multi_step = multi_step_model.predict(val_data_multi, steps=100)[:, 0]
pred_xgb = model_xgb.predict(x_val_multi.reshape(x_val_multi.shape[0], -1))
pred_gru = gru_model.predict(val_ss, steps=100)[:, 0]
pred_transformer = transformer_model.predict(val_ss, steps=100)[:, 0]

# Check for NaNs in predictions
print("NaNs in pred_multi_step:", np.isnan(pred_multi_step).sum())
print("NaNs in pred_xgb:", np.isnan(pred_xgb).sum())
print("NaNs in pred_gru:", np.isnan(pred_gru).sum())
print("NaNs in pred_transformer:", np.isnan(pred_transformer).sum())

# Ensure all predictions have the same length
# Ensure all predictions have the same length
min_length = min(len(pred_multi_step), len(pred_xgb), len(pred_gru), len(pred_transformer))

pred_multi_step = pred_multi_step[:min_length]
pred_xgb = pred_xgb[:min_length]
pred_gru = pred_gru[:min_length]
pred_transformer = pred_transformer[:min_length]

# Combine predictions from all models
combined_forecast = (pred_multi_step + pred_xgb + pred_gru + pred_transformer) / 4

# Check for NaNs in combined forecast
print("NaNs in combined_forecast:", np.isnan(combined_forecast).sum())
# Ensure true values have the same length as combined_forecast
true_values = dataset_scaled[temperature][train_split:train_split + min_length]

# Check for NaNs in true values
print("NaNs in true_values:", np.isnan(true_values).sum())


# Calculate MAE for ensemble model
# true_values = df['temperature'][train_split:train_split + min_length].values
# Remove NaNs from combined forecast and true values
# Remove NaNs from combined forecast and true values
if np.isnan(combined_forecast).sum() > 0 or np.isnan(true_values).sum() > 0:
    valid_indices = ~np.isnan(combined_forecast) & ~np.isnan(true_values)
    combined_forecast = combined_forecast[valid_indices]
    true_values = true_values[valid_indices]

# Calculate MAE for each feature separately
mae_ensemble = np.mean(np.abs(combined_forecast - true_values), axis=0)

# Print MAE for each feature
for i, mae in enumerate(mae_ensemble):
    print(f"MAE for {features_6[i]}: {mae}")

# Optionally, you can calculate the overall MAE by averaging the MAE for all features
overall_mae_ensemble = np.mean(mae_ensemble)
print(f"Overall MAE for Ensemble Model: {overall_mae_ensemble}")



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize the model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Perform Randomized Search
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=50, cv=3, verbose=1, n_jobs=-1)
random_search.fit(x_train_multi.reshape(x_train_multi.shape[0], -1), y_train_multi[:, -1])

# Best parameters
best_params = random_search.best_params_
print(f"Best parameters: {best_params}")


In [ ]:
# Function to evaluate model on test data
def evaluate_model(model, x_test, y_test):
    predictions = model.predict(x_test)
    mae = np.mean(np.abs(predictions - y_test))
    rmse = np.sqrt(np.mean((predictions - y_test)**2))
    return mae, rmse

# Evaluate models
mae_lstm, rmse_lstm = evaluate_model(lstm_model, x_val_uni, y_val_uni)
mae_gru, rmse_gru = evaluate_model(gru_model, x_val_ss, y_val_ss)
mae_transformer, rmse_transformer = evaluate_model(transformer_model, x_val_ss, y_val_ss)
mae_xgb, rmse_xgb = evaluate_model(model_xgb, x_val_multi.reshape(x_val_multi.shape[0], -1), y_val_multi[:, -1])

print(f"LSTM MAE: {mae_lstm}, RMSE: {rmse_lstm}")
print(f"GRU MAE: {mae_gru}, RMSE: {rmse_gru}")
print(f"Transformer MAE: {mae_transformer}, RMSE: {rmse_transformer}")
print(f"XGBoost MAE: {mae_xgb}, RMSE: {rmse_xgb}")


In [ ]:
def plot_forecast(true_values, lstm_pred, gru_pred, transformer_pred, xgb_pred, ensemble_pred):
    plt.figure(figsize=(12, 6))
    plt.plot(true_values, label='True Values')
    plt.plot(lstm_pred, label='LSTM Predictions')
    plt.plot(gru_pred, label='GRU Predictions')
    plt.plot(transformer_pred, label='Transformer Predictions')
    plt.plot(xgb_pred, label='XGBoost Predictions')
    plt.plot(ensemble_pred, label='Ensemble Predictions', linestyle='--')
    plt.legend()
    plt.show()

# Generate predictions for visualization
true_values = y_val_multi[:, -1]
lstm_pred = lstm_model.predict(x_val_multi[:, -1])
gru_pred = gru_model.predict(x_val_ss, steps=100)[:, 0]
transformer_pred = transformer_model.predict(x_val_ss, steps=100)[:, 0]
xgb_pred = model_xgb.predict(x_val_multi.reshape(x_val_multi.shape[0], -1))
# ensemble_pred = ensemble_model.predict(x_val_multi.reshape(x_val_multi.shape[0], -1))

# Plot the forecasts
plot_forecast(true_values, lstm_pred, gru_pred, transformer_pred, xgb_pred)


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# # Assuming you want to use temperature data from x_train
endog = x_train[:, :, 0].reshape(-1)  # Selecting the first column (temperature) as endogenous variable and reshaping to 1D

# # Define SARIMAX model
model = SARIMAX(endog, order=(0, 1, 3), seasonal_order=(0, 1, 1, 12))

# Fit the model
# results = model.fit()
# results = model.fit(maxiter=1000)  # Increase maxiter to 1000 (or higher)
results = model.fit(method='powell')  # Try using the 'powell' method

# Print summary
print(results.summary())


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import warnings

# Define a function to perform grid search for SARIMAX model
# Define a function to perform grid search for SARIMAX model
def sarimax_grid_search(p_values, d_values, q_values, P_values, D_values, Q_values, m_values, train, test):
    # Create a DataFrame to store the results
    column_names = ['p', 'd', 'q', 'P', 'D', 'Q', 'm', 'RMSE_train', 'RMSE_test']
    df = pd.DataFrame(columns=column_names)
    
    # Iterate through all combinations of hyperparameters
    for p in p_values:
        for d in d_values:
            for q in q_values:
                for P in P_values:
                    for D in D_values:
                        for Q in Q_values:
                            for m in m_values:
                                # Fit SARIMAX model
                                try:
                                    model = SARIMAX(train, order=(p, d, q), seasonal_order=(P, D, Q, m))
                                    results = model.fit(method='powell')  # Increase maxiter to 1000 (or higher
                                except Exception as e:
                                    print(f"Error fitting SARIMAX model with parameters ({p}, {d}, {q}), ({P}, {D}, {Q}, {m}): {e}")
                                    continue
                                
                              
                                # Make predictions on train and test datasets
                                train_predictions = pd.Series(results.predict(start=1, end=train_data_len-1), name='Predictions')
                                test_predictions = results.predict(start=len(x_train), end=len(x_train)+len(x_test)-1)

                                
                                # Calculate RMSE values
                                RMSE_train = np.sqrt(mean_squared_error(train_predictions, train[:train_data_len-1]))  # Only consider the first 48 data points for training
                                RMSE_test = np.sqrt(mean_squared_error(test_predictions, test))

                                
                                # Append results to the DataFrame
                                # Append results to the DataFrame
                                new_row = pd.DataFrame({'p': [p], 'd': [d], 'q': [q],
                                                        'P': [P], 'D': [D], 'Q': [Q], 'm': [m],
                                                        'RMSE_train': [RMSE_train], 'RMSE_test': [RMSE_test]})
                                df = pd.concat([df, new_row], ignore_index=True)
    return df


In [ ]:
train = x_train[:, :, 0].reshape(-1)  # Use the temperature data from x_train as the endogenous variable
test = y_test_temp  # Use the temperature data from y_test as the testing dataset
# Define hyperparameters for grid search
# p_values = [0, 1, 2]
# d_values = [0, 1]
# q_values = [0, 1, 2]
# P_values = [0, 1, 2]
# D_values = [0, 1]
# Q_values = [0, 1, 2]
# m_values = [12]

# Define seasonal hyperparameters AND TRY AFTER THE ABOVE 
p = [1,2,3,0]
d = [1,2,3,0]
q = [1,2,3,0]
P = [1,2,3,0]
D = [1,2,3,0] # Seasonal
Q = [1,2,3,0] # Seasonal
m = [12]  # Monthly seasonal cycle

# Disable warnings for cleaner output
warnings.filterwarnings("ignore")

# Perform grid search
grid_results = sarimax_grid_search(p, d, q, P, D, Q, m, endog, test)

# Display grid search results
print(grid_results)

# Save grid search results to a CSV file
grid_results.to_csv("sarimax_grid_search_results.csv")